<a href="https://colab.research.google.com/github/gabie0208/chinese-ltp/blob/main/1128_direction_buyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 6.8 MB/s 
     |████████████████████████████████| 895 kB 41.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import numpy as np
import transformers
from tqdm import tqdm

from transformers import BertTokenizer, BertModel, AdamW
from transformers import BertForMaskedLM,BertForSequenceClassification

import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import random
import time
import datetime

In [3]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
def read_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
        l = []
        for line in lines:
            l.append(line.strip().split("\n"))
    return l

buyu_gilai_test = read_file("/content/qilai-test data.txt")
buyu_qilai_train = read_file("/content/qilai-train data.txt")
buyu_xialai_test = read_file("/content/xialai-test data.txt")
buyu_xialai_train = read_file("/content/xialai-train data.txt")
buyu_xiaqu_test = read_file("/content/xiaqu-test data.txt")
buyu_xiaqu_train = read_file("/content/xiaqu-train data.txt")

In [6]:
buyu_qilai_train

[['啊，我想起来了，去年李成日也托运过。\t2'],
 ['路上要用的东西放在手提包里，这样用起来方便。\t4'],
 ['那件事情说起来容易，做起来很难。\t4'],
 ['看起来不过二十出头。祝你生日快乐！\t4'],
 ['看起来，要下雨了。快走吧！\t4'],
 ['因为送钟跟送终同音，听起来不吉利。\t4'],
 ['看起来，你很像你爸爸。\t4'],
 ['这些菜看起来好像很好吃。\t4'],
 ['我也被举了起来。\t1'],
 ['主人和客人都哈哈大笑起来。\t3'],
 ['说完，放进嘴里吃起来。\t3'],
 ['将做和刚做父亲的男人是最幸福的，记得从妻子刚怀上孩子起，我就兴奋不已，看着她的肚子一天天鼓起来，心里甜滋滋的。\t2'],
 ['调查者分析，被调查青年大都是伴随着改革开放成长起来的一代人，改革时代的艰难和辉煌，使他们的思想状况呈现出强烈的时代色彩，他们的人生价值观趋于功利和现实，把人生关注的目标首先投向个人事业发展、家庭物质生活水平，其次才是为社会进步和国家富强而奋斗。\t2'],
 ['员工经常通宵加班，累了就躺在床垫上休息，醒了爬起来再埋头苦干。\t1'],
 ['她是我的邻居，搬入我们公寓不久，就与大家熟悉起来，成了好朋友。\t3'],
 ['泓庆：不是，昨天晚上，我跟女朋友出去玩儿，回宿舍上楼的时候不小心崴了一下脚，当时没有觉得怎么样，可夜里就开始疼起来了，今天早上一看，脚肿得跟馒头似的，可疼了。\t3'],
 ['这样，孔子又练了起来。\t3'],
 ['所以，虽然是一样地喂养，一样在有客人来时出来介绍一下，然而，如今回想起来，被我们称作“哲学家”的黑黑，在幼年时期，确实是有点被冷落了。\t4'],
 ['一大早，街上便热闹起来。\t3'],
 ['她要中国的妇女不要再依靠男人生活，无论在经济上还是学识上都要独立起来。\t3'],
 ['有一只大蜈蚣风筝，十几个人才能把它放起来。\t2'],
 ['彼得：这雨又下起来了，再坐一会儿吧。\t3'],
 ['进入90年代以来，私人住宅电话已成为一种新的社会消费潮流，消费主体已从少数先富起来的人扩展到工薪阶层，特别是在大城市和沿海经济发达地区，出现了前所未有的“电话消费热”。\t3'],
 ['桑金兰错在场子里走着的时候，却不住地弯下身去，将那些碎毛顺手捡拾起来，塞进斜挎在她腰际

In [15]:
def load_data(data_dir, *filenames):
    sentences = []
    for filename in filenames:
        with open(data_dir + filename, 'r') as f:
            for line in f.readlines():
                sentences.append(line.strip().split('\t')[0])
    return sentences, len(sentences)

In [16]:
data_dir = '/content/'

qilai, total_qilai = load_data(data_dir, "qilai-train data.txt")
xialai, total_xialai = load_data(data_dir, "xialai-train data.txt")
xiaqu, total_xiaqu = load_data(data_dir, "xiaqu-train data.txt")

print("起来 :", total_qilai)
print("下来 :", total_xialai)
print("下去 :", total_xiaqu)

起来 : 5100
下来 : 5101
下去 : 5100


In [17]:
MAX_LEN = 50

def preprocess(data, label):
    sentences = []
    labels = []
    error_cnt = 0
    for sent in data:
        # 해당 문장에 방향보어가 있는 지 확인
        if label not in sent:
            print(f"Sententce : {sent}")
            error_cnt += 1
            continue
        sent = sent[:MAX_LEN]
        sent = sent.replace(label, '[MASK]')
        sentences.append(sent)
        labels.append(label)
    print(f"{label} 방향보어 없는 문장 개수 : {error_cnt}")
    return sentences, labels

In [18]:
qilai_sent, qilai_label= preprocess(qilai, '起来')
xialai_sent, xialai_label = preprocess(xialai, '下来')
xiaqu_sent, xiaqu_label = preprocess(xiaqu, '下去')

起来 방향보어 없는 문장 개수 : 0
下来 방향보어 없는 문장 개수 : 0
下去 방향보어 없는 문장 개수 : 0


In [20]:
all_sent = qilai_sent + xialai_sent + xiaqu_sent
all_label = qilai_label + xialai_label + xiaqu_label

In [21]:
all_sent[:20]

['啊，我想[MASK]了，去年李成日也托运过。',
 '路上要用的东西放在手提包里，这样用[MASK]方便。',
 '那件事情说[MASK]容易，做[MASK]很难。',
 '看[MASK]不过二十出头。祝你生日快乐！',
 '看[MASK]，要下雨了。快走吧！',
 '因为送钟跟送终同音，听[MASK]不吉利。',
 '看[MASK]，你很像你爸爸。',
 '这些菜看[MASK]好像很好吃。',
 '我也被举了[MASK]。',
 '主人和客人都哈哈大笑[MASK]。',
 '说完，放进嘴里吃[MASK]。',
 '将做和刚做父亲的男人是最幸福的，记得从妻子刚怀上孩子起，我就兴奋不已，看着她的肚子一天天鼓[MASK]，心里',
 '调查者分析，被调查青年大都是伴随着改革开放成长[MASK]的一代人，改革时代的艰难和辉煌，使他们的思想状况呈',
 '员工经常通宵加班，累了就躺在床垫上休息，醒了爬[MASK]再埋头苦干。',
 '她是我的邻居，搬入我们公寓不久，就与大家熟悉[MASK]，成了好朋友。',
 '泓庆：不是，昨天晚上，我跟女朋友出去玩儿，回宿舍上楼的时候不小心崴了一下脚，当时没有觉得怎么样，可夜',
 '这样，孔子又练了[MASK]。',
 '所以，虽然是一样地喂养，一样在有客人来时出来介绍一下，然而，如今回想[MASK]，被我们称作“哲学家”的黑黑',
 '一大早，街上便热闹[MASK]。',
 '她要中国的妇女不要再依靠男人生活，无论在经济上还是学识上都要独立[MASK]。']

In [22]:
all_label[:10]

['起来', '起来', '起来', '起来', '起来', '起来', '起来', '起来', '起来', '起来']

In [23]:
train_sent = all_sent
train_label = all_label

In [24]:
MODEL_TYPE = 'bert-base-chinese'
MAX_SIZE = 100
BATCH_SIZE = 100

tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [25]:
tokenized_sent = [tokenizer.tokenize(sent) for sent in train_sent]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_sent]

In [26]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 100
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([1557, 8024, 2769, 2682,  103,  749, 8024, 1343, 2399, 3330, 2768,
       3189,  738, 2805, 6817, 6814,  511,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [27]:
directional_list = ["起来", "下来", "下去"]
labels = [directional_list.index(x) for x in train_label]

In [28]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [29]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=1, test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=1, test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)		

In [30]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [32]:
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=6)
model.cuda()

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [33]:
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW, BertConfig

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [34]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [36]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

In [37]:
# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:02:18

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:05

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.37
  Training epcoh took: 0:02:17

Running Validation...
  Accuracy: 0.85
  Validation took: 0:00:05

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.24
  Training epcoh took: 0:02:17

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:05

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:02:17

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:05

Training complete!


In [38]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [39]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu()

    return logits

In [40]:
logits = test_sentences(['这几天天气突然冷[MASK]了'])
directional_list = ["起来", "下来", "下去"]

print(logits)
print(directional_list[np.argmax(logits)])

tensor([[ 3.4023,  3.0407, -1.3743, -2.8785, -2.8091, -2.6709]])
起来


In [41]:
logits = test_sentences(['教室里突然安静[MASK]了'])
directional_list = ["起来", "下来", "下去"]

print(logits)
print(directional_list[np.argmax(logits)])

tensor([[-1.2115,  5.3776, -0.9689, -2.0435, -2.3364, -1.7642]])
下来


In [42]:
import torch.nn.functional as F

In [43]:
def predict_sentence():
    sent = input('Input Sentence')
    logits = test_sentences([sent])
    softmax = F.softmax(logits[0], dim=0).numpy()
    
    for l, p in zip(directional_list, softmax):
        print(f"{l} : {p*100:.2f}%")

In [44]:
predict_sentence()

Input Sentence教室里突然安静[MASK]了
起来 : 0.14%
下来 : 99.51%
下去 : 0.17%


In [45]:
predict_sentence()

Input Sentence这几天天气突然冷[MASK]了
起来 : 58.44%
下来 : 40.71%
下去 : 0.49%
